# Scheduler K8S Test

Assumes you have a Kind cluster running with metallb. No other dependencies required.
You can use [Seldon Ansible Kind Playbook](https://github.com/SeldonIO/ansible-k8s-collection/blob/master/playbooks/kind.yaml)

## Setup

* `make kind-image-install-all`
* `make deploy` 


In [ ]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## No Auth Example

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

## Auth Preparation - Prepare Minio with Iris Model

Install minio in a kind cluster with Ansible.

Create rclone.conf changing the ip address for minio as appropriate

```
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://172.18.255.1:9000
```
 Download iris model
 
 ```
 cd /tmp
 gsutil cp -r gs://seldon-models/mlserver/iris .
 ```
  
 Copy iris model to minio
 
 ```
 rclone --config ./rclone.conf copy iris s3://models/iris
 ```

## Inline RClone Config Example


In [ ]:
!grpcurl --format text -d '\
         model { \
            meta: { name:"iris"},\
            modelSpec: {uri:"s3://models/iris",\
                        storageConfig: { \
                        storageRcloneConfig: "{\"type\":\"s3\",\"name\":\"s3\",\"parameters\":{\"provider\":\"minio\",\"env_auth\":\"false\",\"access_key_id\":\"minioadmin\",\"secret_access_key\":\"minioadmin\",\"endpoint\":\"http://minio.minio-system:9000\"}}" \
     } \
                        requirements:["sklearn"],\
                        memoryBytes:500},\
            deploymentSpec:{replicas:1}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

## K8S Secret RClone Config Example

 * Update endpoint for minio below

In [ ]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://minio.minio-system:9000

In [ ]:
!kubectl apply -f minio-secret.yaml

In [ ]:
!grpcurl --format text -d '\
         model { \
            meta: { name:"iris"\
                    kubernetesMeta: {namespace: "seldon-mesh"}},\
            modelSpec: {uri:"s3://models/iris",\
                        storageConfig: { \
                          storageSecretName: "minio-secret" \
                         } \
                        requirements:["sklearn"],\
                        memoryBytes:500},\
            deploymentSpec:{replicas:1}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

## Using Agent ConfigMap

 * Edit the AgentConfig map to include the `minio-secret` as one of the rclone defaults, e.g.:
 
 ```
 apiVersion: v1
kind: ConfigMap
metadata:
  name: seldon-agent
data:
  agent.json: |-
    {
       "rclone" : {
           "config_secrets": ["seldon-rclone-gs-public","minio-secret"]
       },
    }

 ```

In [ ]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://minio.minio-system:9000

In [ ]:
!kubectl apply -f minio-secret.yaml

In [ ]:
!grpcurl --format text -d '\
         model { \
            meta: { name:"iris"\
                    kubernetesMeta: {namespace: "seldon-mesh"}},\
            modelSpec: {uri:"s3://models/iris",\
                        requirements:["sklearn"],\
                        memoryBytes:500},\
            deploymentSpec:{replicas:1}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto   ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

# Versions Test

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":50000},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":20000},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris"},"allVersions":true}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis \
        -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model":{"name":"iris"},"allVersions":true}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus